In [2]:
import numpy as np
import pandas as pd
import itertools as it

import math
import joblib

import pickle
#para depurar los que letras que no están en el codigo ascii
import unicodedata
import functools
import spacy

#from spacy_stanfordnlp import StanfordNLPLanguage
from sklearn.decomposition import LatentDirichletAllocation
from sklearn import metrics
#from wiki_dump_reader import Cleaner, iterate
from sklearn.decomposition import TruncatedSVD

from sklearn.preprocessing import normalize
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import pairwise_distances

from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
from sklearn.feature_selection import chi2
from sklearn.decomposition import TruncatedSVD
from sklearn.preprocessing import normalize,scale

import h2o

import pyLDAvis
#import pyLDAvis.gensim  # don't skip this

import gensim
import gensim.corpora as corpora
from gensim.models import CoherenceModel
from gensim.models import HdpModel


from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.decomposition import LatentDirichletAllocation
from multiprocessing import  Pool
import math
import scipy.sparse as sp


from sklearn.model_selection import *
from sklearn.svm import SVC
from sklearn.metrics import classification_report, confusion_matrix

/Users/Raul/miniforge3/envs/kge/lib/python3.10/site-packages/past/builtins/misc.py:45: DeprecationWarning: the imp module is deprecated in favour of importlib and slated for removal in Python 3.12; see the module's documentation for alternative uses
  from imp import reload
/Users/Raul/miniforge3/envs/kge/lib/python3.10/site-packages/gensim/matutils.py:22: DeprecationWarning: Please use `triu` from the `scipy.linalg` namespace, the `scipy.linalg.special_matrices` namespace is deprecated.
  from scipy.linalg.special_matrices import triu


In [ ]:
pd.__version__

# Gesnim

In [3]:
import gensim
import gensim.corpora as corpora
from gensim.corpora import Dictionary
from gensim.models.coherencemodel import CoherenceModel
from gensim.models.ldamodel import LdaModel
import re
import nltk
from pprint import pprint
#textos  = [nltk.word_tokenize(sent) for sent in df.cleaned]
from gensim.parsing.preprocessing import STOPWORDS

In [4]:
from nltk.stem import WordNetLemmatizer
 
from nltk.stem import PorterStemmer
from nltk.tokenize import word_tokenize
  
ps = PorterStemmer()

In [11]:
df = pd.read_csv('../amazon_objects/amazon_raw/amazon_coref_cleaned_ner.csv',sep=';')

In [12]:
nlp = spacy.load('en_core_web_sm', disable=['parser', 'ner'])
pipeline = ["lemmatizer", "tagger", "parser", "ner"]

/Users/Raul/miniforge3/envs/kge/lib/python3.10/site-packages/catalogue/__init__.py:135: DeprecationWarning: SelectableGroups dict interface is deprecated. Use select.
  for entry_point in AVAILABLE_ENTRY_POINTS.get(self.entry_point_namespace, []):
/Users/Raul/miniforge3/envs/kge/lib/python3.10/site-packages/catalogue/__init__.py:135: DeprecationWarning: SelectableGroups dict interface is deprecated. Use select.
  for entry_point in AVAILABLE_ENTRY_POINTS.get(self.entry_point_namespace, []):
/Users/Raul/miniforge3/envs/kge/lib/python3.10/site-packages/catalogue/__init__.py:135: DeprecationWarning: SelectableGroups dict interface is deprecated. Use select.
  for entry_point in AVAILABLE_ENTRY_POINTS.get(self.entry_point_namespace, []):
/Users/Raul/miniforge3/envs/kge/lib/python3.10/site-packages/catalogue/__init__.py:135: DeprecationWarning: SelectableGroups dict interface is deprecated. Use select.
  for entry_point in AVAILABLE_ENTRY_POINTS.get(self.entry_point_namespace, []):
/Users/R

In [13]:
texts = df.clean
docs = list(nlp.pipe(texts))

In [7]:
df.category.unique()

NameError: name 'df' is not defined

In [14]:
# Tags I want to remove from the text
removal= ['ADV','PRON','CCONJ','PUNCT','PART','DET','ADP','SPACE', 'SYM']
tokens = []
for summary in docs:
    proj_tok = [token.lemma_.lower() for token in summary if token.pos_ not in removal and not token.is_stop and token.lemma_.lower() not in STOPWORDS and token.is_alpha]
    tokens.append(proj_tok)

In [16]:
len(tokens)

3000

In [ ]:
#tokens_without_sw = [[re.sub('[^A-Za-z0-9]+', '', word) for word in texto if not lemmatizer.lemmatize(word).lower() in STOPWORDS and re.sub('[^A-Za-z0-9]+', '', word)!=''] for texto in textos ] ## remove also special characters

In [ ]:
#tokens_without_sw = [[word for word in texto if not word.lower() in STOPWORDS ] for texto in textos ] ## remove also special characters

In [17]:
tokens_without_sw = tokens

In [18]:
id2word = corpora.Dictionary(tokens_without_sw)

# Create Corpus

# Term Document Frequency
corpus = [id2word.doc2bow(text) for text in tokens_without_sw]

In [ ]:
# View
lda_model = LdaModel(corpus=corpus,
                   id2word=id2word,
                   num_topics=5, 
                   random_state=0,
                   chunksize=100,
                   alpha='auto',
                   per_word_topics=True)

pprint(lda_model.print_topics())
doc_lda = lda_model[corpus]

In [ ]:
import pyLDAvis
import pyLDAvis.gensim_models


pyLDAvis.enable_notebook()
p = pyLDAvis.gensim_models.prepare(lda_model, corpus, id2word)
p

In [19]:
# Alpha parameter
alpha = list(np.arange(0.01, 1, 0.2))
alpha.append('symmetric')
alpha.append('asymmetric')
# Beta parameter
beta = list(np.arange(0.01, 1, 0.2))
beta.append('symmetric')


In [20]:
def compute_coherence_values(dictionary, corpus, texts, limit, start, step, alpha, beta):
    """
    Compute c_v coherence for various number of topics

    Parameters:
    ----------
    dictionary : Gensim dictionary
    corpus : Gensim corpus
    texts : List of input texts
    limit : Max num of topics

    Returns:
    -------
    model_list : List of LDA topic models
    coherence_values : Coherence values corresponding to the LDA model with respective number of topics
    """
    coherence_values = []
    model_list = []
    model_parameters = []
    for num_topics in range(start, limit, step):
        for a in alpha:
            for b in beta:
                
                model  = LdaModel(corpus=corpus,
                           id2word=id2word,
                           num_topics=num_topics, 
                           random_state=0,
                           chunksize=20,
                           alpha=a,
                                  eta = b,
                           per_word_topics=True)

                model_list.append(model)
                coherencemodel = CoherenceModel(model=model, texts=texts, dictionary=dictionary, coherence='c_v')
                coherence_values.append(coherencemodel.get_coherence())
                model_parameters.append({'num_topics':num_topics,'alpha':a,'eta':b})

    return model_list, coherence_values, model_parameters

In [ ]:
%%time
model_list, coherence_values,model_parameters = compute_coherence_values(dictionary=id2word, corpus=corpus, texts=tokens_without_sw, start=2, limit=20, step=1,alpha = alpha, beta = beta)


In [ ]:
import seaborn as sns

In [ ]:
df = pd.DataFrame(model_parameters)

In [ ]:
df['coherence'] = coherence_values
df.head()

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd

In [ ]:
df ['comparacion'] = 'num_topics'+str(df['num_topics'])+'_alpha_'+str(df['alpha'])+'_eta_'+str(df['eta'])

In [ ]:
df.loc[df.coherence>0.61,:]

In [ ]:
df.loc[((df.num_topics==5) & (df.coherence>=0.55)),:]

In [ ]:
joblib.dump(model_list,'../amazon_objects/lda_model_list_amazon')
joblib.dump(modelo,'../amazon_objects/amazon_topic_modeling_final_gesnim')
joblib.dump(df,'../amazon_objects/amazon_topics_evaluation_models')

In [51]:
import pyLDAvis
import pyLDAvis.gensim_models


pyLDAvis.enable_notebook()
p = pyLDAvis.gensim_models.prepare(modelo, corpus, id2word)
p

/home/raul/kge/lib/python3.7/site-packages/pyLDAvis/_prepare.py:247: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  by='saliency', ascending=False).head(R).drop('saliency', 1)


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
6     -0.123867 -0.137866       1        1  23.175033
1     -0.135820 -0.005340       2        1  20.320682
4     -0.008415 -0.040494       3        1  17.760575
3     -0.157660  0.143963       4        1  17.340536
5      0.133925 -0.100441       5        1  11.330114
0      0.109179  0.103415       6        1   6.509974
2      0.182660  0.036762       7        1   3.563087, topic_info=           Term         Freq        Total Category  logprob  loglift
513        film  1023.000000  1023.000000  Default  30.0000  30.0000
119          mr  2688.000000  2688.000000  Default  29.0000  29.0000
130      people  2485.000000  2485.000000  Default  28.0000  28.0000
115      mobile  1096.000000  1096.000000  Default  27.0000  27.0000
597       blair  1219.000000  1219.000000  Default  26.0000  26.0000
...         ...          ...          ...      ...      ...      ...
2799  criticism    55.460661   101.976606   Topic7  -5.7088   2.7255
378         cup    84.667520   396.909688   Topic7  -5.2858   1.7896
1162     player    82.323956  1000.280167   Topic7  -5.3139   0.8372
372        city    52.672370   252.362216   Topic7  -5.7604   1.7678
2367       ball    50.696128   192.315236   Topic7  -5.7987   2.0013

[420 rows x 6 columns], token_table=      Topic      Freq     Term
term                          
4508      1  0.008070  academy
4508      2  0.008070  academy
4508      3  0.040349  academy
4508      4  0.024209  academy
4508      5  0.008070  academy
...     ...       ...      ...
2995      2  0.012112  zealand
2995      3  0.012112  zealand
2995      4  0.012112  zealand
2995      5  0.944769  zealand
2995      6  0.012112  zealand

[2282 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[7, 2, 5, 4, 6, 1, 3])

In [ ]:
modelo.get_topic_terms(1,10)

In [ ]:
reverse_map = dict((modelo.id2word[id],id) for id in modelo.id2word)

In [59]:
topics = modelo.show_topics(formatted = False, num_words=100)

In [60]:
## HACER DICCIONARIO PARA METER SOLO LO Q ME INTERESA DE LOS TOPICS
tidc = dict()
for topic in topics:
    clave = topic[0]
    lista_terminos = []
    for t in topic[1]:
        lista_terminos.append(t[0])
    tidc[clave]=lista_terminos
joblib.dump(tidc,'./bbc_objects/new_bbc_topics_7_sinprob')

In [60]:
joblib.dump(topics,'new_bbc_topics_7')

['new_bbc_topics_7']

In [61]:
len(topics)

7

In [62]:
diccionario={}
vocabulario = set()

for topic in topics:
    lista_subdocumentos = []
    diccionario1 = dict()
    for tupla in topic[1]:
        vocabulario.add (tupla[0])
        diccionario1[tupla[0]]=tupla[1]
        lista_subdocumentos.append({tupla[0]:tupla[1]})
    diccionario[topic[0]]=diccionario1
        

In [56]:
joblib.dump(diccionario,'new_bbc_topics_7')

['new_bbc_topics_7']

In [57]:
joblib.dump(vocabulario,'vocabulario_7')

['vocabulario_7']

In [ ]:
columnas = [0,1,2,3,4,5,6]
df = pd.DataFrame(columns=['0','1','2','3','4','5','6'])
cont = 0
for documento in modelo.get_document_topics(corpus):
    visitados = []
    listado_valores = []
    print ('visitando el doc '+str(cont))
    print (documento)
    for tupla in documento:
        actual = tupla[0]# mejor coger min y max y rellenar con 0
        print('actual '+str(actual))
        if (len(visitados)>0):
            last_visited = visitados[len(visitados)-1]
        else:
            last_visited = -1
        visitados.append(actual)
        visitados.sort()
        print('ultimo '+str(last_visited))

        if actual == 0  or (actual>0 and actual - last_visited==1):
            listado_valores.append(tupla[1])
        elif actual>0 and actual - last_visited!=1:
            for i in range(last_visited+1,actual):
                visitados.append(i)
                listado_valores.append(0)
                visitados.sort()
            listado_valores.append(tupla[1])
    last_visited = visitados[len(visitados)-1]
    print(visitados)
    if 6 not in visitados:
        
        for i in range(last_visited+1,7):
            visitados.append(i)
            listado_valores.append(0)
    cont = cont + 1
    print(listado_valores)
    df.loc[len(df)] = listado_valores
    
 

In [69]:
df = df.loc[:,[column for column in df.columns if column!='target']]

In [70]:
df.head()

,0,1,2,3,4,5,6
0,0.000000,0.025334,0.000000,0.791117,0.178134,0.000000,0.000000
1,0.000000,0.490732,0.019887,0.018978,0.000000,0.038238,0.430617
2,0.000000,0.000000,0.045011,0.000000,0.348087,0.484244,0.118659
3,0.000000,0.026996,0.572469,0.000000,0.072849,0.325870,0.000000
4,0.628837,0.000000,0.000000,0.000000,0.314380,0.053325,0.000000


In [ ]:
topics = []

for indx,x in df.iterrows():
    print(x)
    topics.append(np.argmax(x))

df['new_topic']=topics

In [72]:
df.head()

,0,1,2,3,4,5,6,new_topic
0,0.000000,0.025334,0.000000,0.791117,0.178134,0.000000,0.000000,3
1,0.000000,0.490732,0.019887,0.018978,0.000000,0.038238,0.430617,1
2,0.000000,0.000000,0.045011,0.000000,0.348087,0.484244,0.118659,5
3,0.000000,0.026996,0.572469,0.000000,0.072849,0.325870,0.000000,2
4,0.628837,0.000000,0.000000,0.000000,0.314380,0.053325,0.000000,0


In [73]:
joblib.dump(df,'doc_topic_matrix_bbc_7')

['doc_topic_matrix_bbc_7']

In [74]:
df2 = joblib.load('bbc_coref_resolved_cleaned')
df2.head()

,category,text,coref_text,cleaned
0,tech,tv future in the hands of viewers with home th...,tv future in the hands of viewers with home th...,tv future hand viewer home theatre system plas...
1,business,worldcom boss left books alone former worldc...,worldcom boss left books alone former worldc...,worldcom boss leave book worldcom boss bernie ...
2,sport,tigers wary of farrell gamble leicester say ...,tigers wary of farrell gamble leicester say ...,tiger wary farrell gamble leicester tiger wary...
3,sport,yeading face newcastle in fa cup premiership s...,yeading face newcastle in fa cup premiership s...,yeade face newcastle fa cup premiership newcas...
4,entertainment,ocean s twelve raids box office ocean s twelve...,ocean s twelve raids box office ocean s twelve...,ocean s raid box office ocean s crime caper se...


In [78]:
df['target']=df2.category
df['target'] = df['target'].astype('category')

In [79]:
df.shape

(2225, 9)

In [80]:
df.head()

,0,1,2,3,4,5,6,new_topic,target
0,0.000000,0.025334,0.000000,0.791117,0.178134,0.000000,0.000000,3,tech
1,0.000000,0.490732,0.019887,0.018978,0.000000,0.038238,0.430617,1,business
2,0.000000,0.000000,0.045011,0.000000,0.348087,0.484244,0.118659,5,sport
3,0.000000,0.026996,0.572469,0.000000,0.072849,0.325870,0.000000,2,sport
4,0.628837,0.000000,0.000000,0.000000,0.314380,0.053325,0.000000,0,entertainment


In [81]:
df.columns = ['0','1','2','3','4','5','6','new_topic','target']
df.to_csv('./datatopic_bbc_7.csv',index=False)

In [6]:
import h2o
from h2o.estimators import H2OXGBoostEstimator, H2ODeepLearningEstimator,H2ORandomForestEstimator,H2OSupportVectorMachineEstimator
h2o.init(max_mem_size='14G')

Checking whether there is an H2O instance running at http://localhost:54321 ..... not found.
Attempting to start a local H2O server...
  Java Version: openjdk version "1.8.0_312"; OpenJDK Runtime Environment (build 1.8.0_312-8u312-b07-0ubuntu1~20.04-b07); OpenJDK 64-Bit Server VM (build 25.312-b07, mixed mode)
  Starting server from /home/raul/kge/lib/python3.7/site-packages/h2o/backend/bin/h2o.jar
  Ice root: /tmp/tmpk_bvbxb3
  JVM stdout: /tmp/tmpk_bvbxb3/h2o_raul_started_from_python.out
  JVM stderr: /tmp/tmpk_bvbxb3/h2o_raul_started_from_python.err
  Server is running at http://127.0.0.1:54321
Connecting to H2O server at http://127.0.0.1:54321 ... successful.


H2O_cluster_uptime:,01 secs
H2O_cluster_timezone:,Europe/Madrid
H2O_data_parsing_timezone:,UTC
H2O_cluster_version:,3.36.0.3
H2O_cluster_version_age:,1 month and 17 days
H2O_cluster_name:,H2O_from_python_raul_qfsr0t
H2O_cluster_total_nodes:,1
H2O_cluster_free_memory:,12.44 Gb
H2O_cluster_total_cores:,12
H2O_cluster_allowed_cores:,12
H2O_cluster_status:,"locked, healthy"


In [82]:
frame = h2o.upload_file('./datatopic_bbc_7.csv',sep=',',header=1)

Checking whether there is an H2O instance running at http://localhost:54321 ..... not found.
Attempting to start a local H2O server...
  Java Version: openjdk version "1.8.0_312"; OpenJDK Runtime Environment (build 1.8.0_312-8u312-b07-0ubuntu1~20.04-b07); OpenJDK 64-Bit Server VM (build 25.312-b07, mixed mode)
  Starting server from /home/raul/kge/lib/python3.7/site-packages/h2o/backend/bin/h2o.jar
  Ice root: /tmp/tmpiqv3ptji
  JVM stdout: /tmp/tmpiqv3ptji/h2o_raul_started_from_python.out
  JVM stderr: /tmp/tmpiqv3ptji/h2o_raul_started_from_python.err
  Server is running at http://127.0.0.1:54321
Connecting to H2O server at http://127.0.0.1:54321 ... successful.


H2O_cluster_uptime:,01 secs
H2O_cluster_timezone:,Europe/Madrid
H2O_data_parsing_timezone:,UTC
H2O_cluster_version:,3.36.0.3
H2O_cluster_version_age:,1 month and 10 days
H2O_cluster_name:,H2O_from_python_raul_taih7y
H2O_cluster_total_nodes:,1
H2O_cluster_free_memory:,12.44 Gb
H2O_cluster_total_cores:,12
H2O_cluster_allowed_cores:,12
H2O_cluster_status:,"locked, healthy"


Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%


In [83]:
frame['new_topic']=frame['new_topic'].asfactor()
columnas =list( df.columns )
columnas.remove('target')
columnas.remove('new_topic')


In [84]:
frame.head()

0,1,2,3,4,5,6,new_topic,target
0,0.0253343,0,0.791117,0.178134,0,0,3,tech
0,0.490732,0.0198867,0.0189784,0,0.0382377,0.430617,1,business
0,0,0.0450105,0,0.348087,0.484244,0.118659,5,sport
0,0.0269959,0.572469,0,0.0728495,0.32587,0,2,sport
0.628837,0,0,0,0.31438,0.0533246,0,0,entertainment
0,0,0,0,0.0935716,0,0.904415,6,politics
0,0.0470195,0,0,0.192718,0,0.755944,6,politics
0.0950822,0,0.0403145,0,0,0.85952,0,5,sport
0.0461534,0,0,0,0,0.943611,0,5,sport
0.477958,0.0745741,0,0,0.377126,0,0.0659612,0,entertainment


In [85]:

from h2o.estimators.xgboost import H2OXGBoostEstimator
from h2o.estimators import H2OXGBoostEstimator
from h2o.grid.grid_search import H2OGridSearch

search_criteria={'strategy':"RandomDiscrete",
                'max_models':15,
                'seed':4241}

# Rango de hiper-parámetros para grid search 9 combinaciones 
hyper_params = {'sample_rate':[0.4,0.5,0.6]  , 
                'col_sample_rate':[0.4,0.5,0.6],
                'min_rows':[30,50],
               'max_depth':[10,30,100],
               'ntrees':[10,20,30,100,150],
                'reg_lambda':[0,0.2,0.5,0.7,1] ,
                'reg_alpha':[0,0.2,0.5,0.7,1] } 

xgb_rand_grid=H2OGridSearch(
#             H2OGradientBoostingEstimator(
                H2OXGBoostEstimator(
                model_id='gbm_rand_grid',
                seed=665,
                # ntrees=150,
                nfolds=5, #2,
                stopping_metric='mean_per_class_error',
                stopping_rounds=10,
                score_tree_interval=1
                
            ),
    search_criteria=search_criteria,
    hyper_params=hyper_params)

      
xgb_rand_grid.train(x = columnas,y='target',training_frame=frame)

xgboost Grid Build progress: |███████████████████████████████████████████████████| (done) 100%
      col_sample_rate  max_depth  min_rows  ntrees  reg_alpha  reg_lambda  \
0                 0.6       10.0      30.0    36.0        0.0         0.5   
1                 0.4      100.0      30.0    48.0        0.2         1.0   
2                 0.6      100.0      30.0    20.0        0.7         1.0   
3                 0.6      100.0      30.0    27.0        0.7         0.2   
4                 0.6       30.0      30.0    20.0        0.5         0.7   
5                 0.5      100.0      50.0    20.0        1.0         0.0   
6                 0.5       10.0      50.0    20.0        0.7         0.2   
7                 0.5       30.0      50.0    36.0        0.5         0.2   
8                 0.5       30.0      50.0    29.0        0.2         0.5   
9                 0.6      100.0      50.0    29.0        0.5         0.0   
10                0.4       30.0      50.0    37.0        

In [86]:
xgb_rand_grid[0]

Model Details
H2OXGBoostEstimator :  XGBoost
Model Key:  Grid_XGBoost_py_1_sid_bec4_model_python_1648380992064_1_model_13


Model Summary: 


,,number_of_trees
0,,36.0




ModelMetricsMultinomial: xgboost
** Reported on train data. **

MSE: 0.04432971878755898
RMSE: 0.21054623907246356
LogLoss: 0.15930883955198874
Mean Per-Class Error: 0.05043530826650629
AUC: NaN
AUCPR: NaN
Multinomial auc values: Table is not computed because it is disabled (model parameter 'auc_type' is set to AUTO or NONE) or due to domain size (maximum is 50 domains).
Multinomial auc_pr values: Table is not computed because it is disabled (model parameter 'auc_type' is set to AUTO or NONE) or due to domain size (maximum is 50 domains).

Confusion Matrix: Row labels: Actual class; Column labels: Predicted class


,business,entertainment,politics,sport,tech,Error,Rate
0,486.0,5.0,10.0,4.0,5.0,0.047059,24 / 510
1,6.0,361.0,5.0,5.0,9.0,0.064767,25 / 386
2,17.0,5.0,393.0,1.0,1.0,0.057554,24 / 417
3,0.0,13.0,0.0,498.0,0.0,0.025440,13 / 511
4,7.0,13.0,3.0,0.0,378.0,0.057357,23 / 401
5,516.0,397.0,411.0,508.0,393.0,0.048989,109 / 2.225



Top-5 Hit Ratios: 


...

,,mean,sd,cv_1_valid,cv_2_valid,cv_3_valid,cv_4_valid,cv_5_valid
0,accuracy,0.933461,0.019182,0.921348,0.932018,0.956044,0.909297,0.948598
1,auc,NaN,0.000000,NaN,NaN,NaN,NaN,NaN
2,err,0.066539,0.019182,0.078652,0.067982,0.043956,0.090703,0.051402
3,err_count,29.600000,8.502941,35.000000,31.000000,20.000000,40.000000,22.000000
4,logloss,0.196791,0.013164,0.192673,0.209961,0.176960,0.207250,0.197110
5,max_per_class_error,0.101288,0.018677,0.095890,0.123077,0.079545,0.118182,0.089744
6,mean_per_class_accuracy,0.931813,0.019969,0.919195,0.925218,0.957036,0.909596,0.948019
7,mean_per_class_error,0.068187,0.019969,0.080805,0.074782,0.042965,0.090404,0.051981
8,mse,0.055289,0.006394,0.056142,0.058669,0.045356,0.062410,0.053871
9,pr_auc,NaN,0.000000,NaN,NaN,NaN,NaN,NaN



Scoring History: 


,,timestamp,duration,number_of_trees,training_rmse,training_logloss,training_classification_error,training_auc,training_pr_auc
0,,2022-03-27 13:37:02,10.750 sec,0.0,0.800000,1.609438,0.770337,NaN,NaN
1,,2022-03-27 13:37:02,10.757 sec,1.0,0.655261,1.073837,0.065169,NaN,NaN
2,,2022-03-27 13:37:02,10.762 sec,2.0,0.558596,0.828528,0.072360,NaN,NaN
3,,2022-03-27 13:37:02,10.767 sec,3.0,0.472935,0.647784,0.071011,NaN,NaN
4,,2022-03-27 13:37:02,10.772 sec,4.0,0.404893,0.518344,0.058876,NaN,NaN
5,,2022-03-27 13:37:02,10.777 sec,5.0,0.354678,0.428449,0.062472,NaN,NaN
6,,2022-03-27 13:37:02,10.782 sec,6.0,0.313658,0.357414,0.060674,NaN,NaN
7,,2022-03-27 13:37:02,10.787 sec,7.0,0.289258,0.313576,0.062472,NaN,NaN
8,,2022-03-27 13:37:02,10.791 sec,8.0,0.264673,0.268932,0.058427,NaN,NaN
9,,2022-03-27 13:37:02,10.798 sec,9.0,0.252384,0.244491,0.057978,NaN,NaN



See the whole table with table.as_data_frame()

Variable Importances: 


,variable,relative_importance,scaled_importance,percentage
0,1,1748.155884,1.000000,0.238500
1,6,1548.594116,0.885844,0.211273
2,3,1282.769531,0.733784,0.175007
3,5,1214.459351,0.694709,0.165688
4,0,819.560608,0.468814,0.111812
5,4,548.626892,0.313832,0.074849
6,2,167.642563,0.095897,0.022871


In [87]:
xgb_rand_grid[0].save_mojo('bbc_evaluation_topics_model')

'/home/raul/doctorado/notebooks/semantic_oboe/notebooks/bbc/bbc_evaluation_topics_model/Grid_XGBoost_py_1_sid_bec4_model_python_1648380992064_1_model_13.zip'

In [1]:
import joblib
import pandas as pd

In [8]:
corpus = joblib.load('./bbc_objects/bbc_coref_resolved_cleaned')
df = joblib.load('./bbc_objects/doc_topic_matrix_bbc_7')

In [9]:
corpus['new_target'] = df['new_topic']

In [10]:
corpus.head()

,category,text,coref_text,cleaned,entidades,new_target
0,tech,tv future in the hands of viewers with home th...,tv future in the hands of viewers with home th...,tv future hand viewer home theatre system plas...,"{nine months to a years , today, one, five ye...",3
1,business,worldcom boss left books alone former worldc...,worldcom boss left books alone former worldc...,worldcom boss leave book worldcom boss bernie ...,"{last two months, about $180bn, 160bn, the lat...",1
2,sport,tigers wary of farrell gamble leicester say ...,tigers wary of farrell gamble leicester say ...,tiger wary farrell gamble leicester tiger wary...,"{five weeks ago, another three months}",5
3,sport,yeading face newcastle in fa cup premiership s...,yeading face newcastle in fa cup premiership s...,yeade face newcastle fa cup premiership newcas...,"{third, second, saturday, one, the weekend, tw...",2
4,entertainment,ocean s twelve raids box office ocean s twelve...,ocean s twelve raids box office ocean s twelve...,ocean s raid box office ocean s crime caper se...,"{third, one, $38.1m, fourth, twelve, last week...",0


In [11]:
joblib.dump(corpus,'bbc_objects/bbc_processed_final')

['bbc_objects/bbc_processed_final']